In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
from tensorflow.keras import layers
import os
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.models import Sequential
from tensorflow.python.keras import backend as K
from collections import Counter
import random

In [2]:
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.11.0
Num GPUs Available:  0


In [3]:
# Load the dataset without any predefined batch size
train_data = tf.keras.utils.image_dataset_from_directory(
    'images/train',
    labels='inferred',
    label_mode='int',
    batch_size=None,  # Load as unbatched dataset
    image_size=(48, 48),  # Resize images to standard size
    seed=42 
)

Found 28709 files belonging to 7 classes.


In [4]:
test_data = keras.utils.image_dataset_from_directory(
    directory = 'images/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 120,
    image_size = (48, 48),
    seed=42 
)

Found 7178 files belonging to 7 classes.


In [5]:
# Class names
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [6]:
# Extract data from train dataset
def extract_images_and_labels(dataset):
    X, y = [], []
    for image, label in dataset:
        X.append(image.numpy())
        y.append(label.numpy())
    return np.array(X), np.array(y)

# Assuming train_data is loaded correctly
X, y = extract_images_and_labels(train_data)

2024-12-07 21:31:06.337529: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [7]:
print(f'X shape before augmentation: {X.shape}')
print(f'y shape before augmentation: {y.shape}')

X shape before augmentation: (28709, 48, 48, 3)
y shape before augmentation: (28709,)


In [8]:
def data_augmentation(X, y, target_count=2000, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                      zoom_range=0.1, brightness_range=(0.95, 1.05), horizontal_flip=True, vertical_flip=True, fill_mode='nearest'):
    # Initialize ImageDataGenerator
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        zoom_range=zoom_range,
        brightness_range=brightness_range,
        horizontal_flip=horizontal_flip,
        vertical_flip=vertical_flip,
        fill_mode=fill_mode
    )

    balanced_X = []
    balanced_y = []
    
    unique_classes = np.unique(y)
    # For each class, augment until reaching the target count
    for class_label in unique_classes:
        class_indices = np.where(y == class_label)[0]
        class_images = X[class_indices]
        class_labels = y[class_indices]
        num_images = class_images.shape[0]

        # Calculate how many augmentations are needed
        augmentations_needed = target_count - num_images

        # Add original images to balanced dataset
        balanced_X.extend(class_images)
        balanced_y.extend(class_labels)

        # Calculate how many augmentations are needed for this class
        augmentations_needed = target_count - num_images

        # Augment images until we reach the target count
        while augmentations_needed > 0:
            for img, label in zip(class_images, class_labels):
                if augmentations_needed <= 0:
                    break
                img = img.reshape((1,) + img.shape)  # Reshape to (1, height, width, channels)
                label = label.reshape((1,))  # Reshape to a single label
                augmented_img = next(datagen.flow(img))  # Generate an augmented image
                balanced_X.append(augmented_img.squeeze())  # Add augmented image to list
                balanced_y.append(label.squeeze())  # Add label to list
                augmentations_needed -= 1

    balanced_X = np.array(balanced_X)
    balanced_y = np.array(balanced_y)

    return balanced_X, balanced_y

In [9]:
X_augmented, y_augmented = data_augmentation(X, y, target_count=2000)

In [10]:
print(f'X shape after augmentation: {X_augmented.shape}')
print(f'y shape after augmentation: {y_augmented.shape}')

X shape after augmentation: (30273, 48, 48, 3)
y shape after augmentation: (30273,)


In [11]:
Counter(y_augmented)

Counter({3: 7215, 4: 4965, 5: 4830, 2: 4097, 0: 3995, 6: 3171, 1: 2000})

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit
X = np.array(X_augmented)  # Images
y = np.array(y_augmented)  # Labels

In [13]:
# Parameters for stratified shuffle split
batch_size = 126
n_splits = 1  # Number of splits you want
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=None, train_size=batch_size / len(y))

In [14]:
# Generate stratified batches
batches = []
for _, batch_indices in sss.split(X, y):
    batch_images = X[batch_indices].astype(np.float32)/ 255.0  # Ensure images are float32
    batch_labels = y[batch_indices].astype(np.int32)    # Ensure labels are int32
    batches.append((batch_images, batch_labels))

In [15]:
# Convert to TensorFlow Dataset
final_dataset = tf.data.Dataset.from_tensor_slices(
    (
        tf.convert_to_tensor([batch[0] for batch in batches], dtype=tf.float32),  # Images as float32
        tf.convert_to_tensor([batch[1] for batch in batches], dtype=tf.int32),   # Labels as int32
    )
)

In [16]:
# Inspect the first batch
for batch in final_dataset.take(1):
    print(f"Image batch shape: {batch[0].shape}, Label batch shape: {batch[1].shape}")
    print(batch[1])

Image batch shape: (30147, 48, 48, 3), Label batch shape: (30147,)
tf.Tensor([3 6 2 ... 3 0 2], shape=(30147,), dtype=int32)


In [17]:
# Split into training and validation sets
validation_split = 0.2
num_batches = len(batches)

val_dataset = final_dataset.take(int(num_batches * validation_split))
train_dataset = final_dataset.skip(int(num_batches * validation_split))

In [18]:
num_classes = 7

In [19]:
model_file = "my_model.keras"

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [21]:
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 18, 18, 64)        36928     
                                                        

In [ ]:
history = model.fit(train_dataset, epochs=50, validation_data=val_dataset)

Epoch 1/50


In [ ]:
model.save(model_file)

In [ ]:
plt.plot(history.history['accuracy'], color = 'red', label = 'train')
plt.plot(history.history['val_accuracy'], color = 'blue', label = 'validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], color = 'red', label = 'train')
plt.plot(history.history['val_loss'], color = 'blue', label = 'validation')
plt.legend()
plt.show()

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")